In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('cnn_dailymail', '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /Users/nitec427/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print("Features ", dataset['train'].column_names )

Features  ['article', 'highlights', 'id']


In [7]:
len(dataset['train'][1])
sample = dataset['train'][1]
len(sample['article'])

4051

In [8]:
sample['article'][:500]

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s'

In [10]:
sample_text = dataset['train'][1]['article'][:2000]

In [11]:
sample_text

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [12]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/nitec427/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
string = "This is a string. The U.N is a big organization."
sent_tokenize(string)

['This is a string.', 'The U.N is a big organization.']

In [14]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [15]:
summaries = {}
summaries['baseline'] = three_sentence_summary(sample_text)

In [16]:
from transformers import pipeline, set_seed
set_seed(42)
pipe = pipeline("text-generation", model='gpt2-xl')
gpt2_query = sample_text + '\nTL;DR:\n'
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries['gpt2'] = '\n'.join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [17]:
summaries

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."',
 'gpt2': 'First of all, the tenth floor.\nIt\'s called the "forgotten floor."\nIt\'s the "forgotten" because the mentally ill can\'t be placed there.\nThey need more services, but they\'re not getting any.\nThey\'re incarcerated in the building where they committed their crime.\nThe system doesn\'t have the funds to provide for them.\nThey don\'t get their medications.'}

In [18]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)

/Users/nitec427/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]['summary_text']))

In [23]:
# summaries, print prrettily
for k,v in sorted(summaries.items()):
    print(k)
    print(v)
    print()

baseline
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

gpt2
First of all, the tenth floor.
It's called the "forgotten floor."
It's the "forgotten" because the mentally ill can't be placed there.
They need more services, but they're not getting any.
They're incarcerated in the building where they committed their crime.
The system doesn't have the funds to provide for them.
They don't get their medications.

t5
mentally ill inmates are housed on the ninth floor of a florida jail .
most face drug charges or charges of assaulting an officer .
judge says arrests often resu

In [24]:
pipe = pipeline("summarization", model='facebook/bart-large-cnn')
pipe_out = pipe(sample_text)
summaries['bart'] = '\n'.join(pipe_out[0]['summary_text'])

In [28]:
# show where hugging face models are stored
import transformers
print('hugging face models are stored in:', transformers.__path__)
transformers.__path__[0]

hugging face models are stored in: ['/Users/nitec427/miniconda3/envs/torch/lib/python3.10/site-packages/transformers']


'/Users/nitec427/miniconda3/envs/torch/lib/python3.10/site-packages/transformers'